In [ ]:
#creating a spark instance
from pyspark.sql import SparkSession
spark = SparkSession .builder.appName('removeduplicatescart').getOrCreate()

In [ ]:
#import required python libraries
import pandas as pd
import numpy as np
from pyspark.sql import functions as sf

In [ ]:
#Column names and data are in two separate tsv files
#Read column names from tsv file
headers1 = spark.read.csv("sample1.csv",inferSchema =True, header=True)
#headers1.printSchema()

In [ ]:
#Read data from tsv file
rows = spark.read.option("sep", "\t").csv("..\hit_data.tsv",inferSchema =True, header=False)
#rows.printSchema()

In [ ]:
#adding headers to data
rowsWithHeader1 = headers1.union(rows)
#rowsWithHeader1.printSchema()
rowsWithHeader1.count()

In [ ]:
#According to adobe clickstream data feeds page 52, Identifying a unique visitor
#Removing rows with exclude_hit value <= 0
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.exclude_hit<=0)
rowsCount = rowsWithHeader1.count()
rowsCount

In [ ]:
#filtering rows based on hit_source column value. Exclude all rows with hit_source = 5,7,8,9.
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=5)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=7)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=8)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=9)

In [ ]:
#Selecting only post columns; The post column contains the value after processing.
postData = rowsWithHeader1.select(*filter(lambda col: 'post_' in col,rowsWithHeader1.columns))
#Printing no. of columns
len(postData.columns)

In [ ]:
#Adding a new column which conains values to idenify unique visitors
postDataWithUniqueId = postData.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#apart from post_ columns considering column obtained after joining the two post_ columns as mentioned above and mcvisid
postDataWithUniqueId = postDataWithUniqueId.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
#Select the column post_uniqueID
UniqueId=postDataWithUniqueId.select('post_uniqueId')
#Print count
UniqueId.count()

In [ ]:
postDataWithUniqueId.columns

In [ ]:
#Remove post_visid_high, post_visid_low after obtaining unique id
postDataWithUniqueId = postDataWithUniqueId.drop('post_visid_high','post_visid_low')

In [ ]:
#Printing schema of dataframe
postDataWithUniqueId.schema

In [ ]:
#select users with cart and dell.com
cartDellRows = postDataWithUniqueId.filter(postDataWithUniqueId.post_prop14.rlike("^(.)*(dell.com)(.)*(cart)(.)*$"))
cartDellRows.count()

In [ ]:
#storing uniqueids of all existing distinct users
uniqueIdsWithCartDell = [list(x.asDict().values())[0] for x in cartDellRows.select("post_uniqueid").distinct().collect()]
print("{}".format(len(uniqueIdsWithCartDell)))

In [ ]:
#Get all rows of users having cart and dell.com in prop14 column
allRowsOfCartDellUsers = postDataWithUniqueId.where(postDataWithUniqueId.post_uniqueId.isin(uniqueIdsWithCartDell))
#Print count
allRowsOfCartDellUsersCount = allRowsOfCartDellUsers.count()
allRowsOfCartDellUsersCount

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
event1CartDellRows = allRowsOfCartDellUsers.filter(allRowsOfCartDellUsers.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#print count
event1CartDellRows.count()

In [ ]:
#Get unique Ids
event1CartDellUniqueIds = [list(x.asDict().values())[0] for x in event1CartDellRows.select("post_uniqueid").distinct().collect()]
#print count
print("{}".format(len(event1CartDellUniqueIds)))

In [ ]:
#Get abandoned userIds
abandonedCartDellUsersIds = list(set(uniqueIdsWithCartDell) - set(event1CartDellUniqueIds))
#print count
print("{}".format(len(abandonedCartDellUsersIds)))

In [ ]:
#Reducing data by removing null columns
allRowsOfCartDellUsersPDF = allRowsOfCartDellUsers.toPandas()
allRowsOfCartDellUsersPDF.shape

In [ ]:
#Removes any columns with null values
rowsWithoutNullColumns = allRowsOfCartDellUsersPDF.dropna(axis=1,how='all')
rowsWithoutNullColumns.shape

In [ ]:
#Removing columns with 50% nulls 
thre = allRowsOfCartDellUsersCount/2 #Divide by 4 for 25%
rowsWithoutNullColumnsThresh50 = rowsWithoutNullColumns.dropna(axis=1,thresh = thre)
rowsWithoutNullColumnsThresh50.shape

In [ ]:
#Save dataframe to a tsv file
rowsWithoutNullColumnsThresh50.to_csv("version1/reducedusersrows.tsv", sep="\t", index=False)

In [ ]:
#Read tsv file
allUsers = spark.read.option("sep", "\t").csv("version1/reducedusersrows.tsv",inferSchema =True, header=True)
#allUsers.printSchema()

In [ ]:
#Printing column in dataframe
allUsers.schema.names

In [ ]:
#get all rows of abandoned users
allRowsOfAbandonedUsers = allUsers.where(allUsers.post_uniqueId.isin(abandonedCartDellUsersIds))
#print count
allRowsOfAbandonedUsers.count()

In [ ]:
#Groupby abandoned users using post_uniqueId
abandonedUsersGroupBy = allRowsOfAbandonedUsers.groupBy("post_uniqueId").count()

In [ ]:
#Displaying grouped data of abandoned users
abandonedUsersGroupBy.show()

In [ ]:
#get all rows of purchased users
allRowsOfPurchasedUsers = allUsers.where(allUsers.post_uniqueId.isin(event1CartDellUniqueIds))
#print count
allRowsOfPurchasedUsers.count()

In [ ]:
#Groupby purchased users using post_uniqueId
purchasedUsersGroupBy = allRowsOfPurchasedUsers.groupBy("post_uniqueId").count()

In [ ]:
#Displaying grouped data of purchased users
purchasedUsersGroupBy.show()

In [ ]:
#Converting to pandas Dataframe
purchasedPDF = allRowsOfPurchasedUsers.toPandas()

In [ ]:
#print the dataframe
purchasedPDF

In [ ]:
#print column datatypes
purchasedPDF.dtypes

In [ ]:
#fill na values with none
purchasedPDFFill = purchasedPDF.fillna('none')

In [ ]:
#Sort by user id
purchasedPDFFill = purchasedPDFFill.sort_values(by=['post_uniqueId'])
purchasedPDFFill = purchasedPDFFill.reset_index(drop=True)

In [ ]:
#Convert all columns to string type
purchasedPDFFillString = purchasedPDFFill.applymap(str)
purchasedPDFFillString.dtypes

In [ ]:
#create an empty dataframe
purchasedSingleRow = pd.DataFrame()

In [ ]:
#Storing uniqueid
purchasedSingleRow['post_uniqueId'] = purchasedPDFFillString.groupby(['post_uniqueId'])['post_uniqueId'].count().index.get_level_values('post_uniqueId')
#purchasedSingleRow is a dataframe containing post_uniqueId column

In [ ]:
#Getting browser height of each user
h = purchasedPDFFill.groupby(['post_uniqueId'])['post_browser_height'].apply(lambda x: x.max()).reset_index()
h['post_uniqueId'] = h['post_uniqueId'].astype(str)
h = h.sort_values(by=['post_uniqueId'])
h = h.reset_index(drop=True)

In [ ]:
#Getting browser width of each user
w = purchasedPDFFill.groupby(['post_uniqueId'])['post_browser_width'].apply(lambda x: x.max()).reset_index()
w['post_uniqueId'] = w['post_uniqueId'].astype(str)
w = w.sort_values(by=['post_uniqueId'])
w = w.reset_index(drop=True)

In [ ]:
#Get time spent by user
diff = purchasedPDFFill.groupby(['post_uniqueId'])['post_cust_hit_time_gmt'].apply(lambda x: x.max()-x.min()).reset_index()

In [ ]:
#sorting the data by post_uniqueId values
diff['post_uniqueId'] = diff['post_uniqueId'].astype(str)
diff = diff.sort_values(by=['post_uniqueId'])
diff = diff.reset_index(drop=True)

In [ ]:
#adding new column timespent
purchasedSingleRow['timespent'] = diff['post_cust_hit_time_gmt']

In [ ]:
#Converting multiple rows of data into single row by joining the values using ~ for purchased users
for cols in purchasedPDFFillString.columns:
    if (cols != 'post_uniqueId'):
        temp = purchasedPDFFillString.groupby(['post_uniqueId'])[cols].apply(lambda x: '~'.join(x)).reset_index()
        purchasedSingleRow[cols] = temp[cols]

In [ ]:
#replacing post_browser_height with single value
purchasedSingleRow['post_browser_height'] = h['post_browser_height']

In [ ]:
#replacing post_browser_width with single value
purchasedSingleRow['post_browser_width'] = w['post_browser_width']

In [ ]:
#Get count of number times cart was accessed
import re
purchasedSingleRow['cartcount'] = purchasedSingleRow['post_prop14'].apply(lambda x: len(re.findall("cart", x)))

In [ ]:
#Get count of total no. of events performed by user
purchasedSingleRow['eventcount'] = purchasedSingleRow['post_event_list'].apply(lambda x: len(re.findall(",", x))+len(re.findall("~", x))+1)

In [ ]:
#1 if user purchased 0 if abandoned
purchasedSingleRow['purchased'] = 1

In [ ]:
#Displaying newly added columns
purchasedSingleRow[['cartcount','eventcount','timespent','purchased']]

In [ ]:
#Convert to pandas Dataframe
abandonedPDF = allRowsOfAbandonedUsers.toPandas()

In [ ]:
#print
abandonedPDF

In [ ]:
#fill na values with none
abandonedPDFFill = abandonedPDF.fillna('none')

In [ ]:
#Sort by user id
abandonedPDFFill = abandonedPDFFill.sort_values(by=['post_uniqueId'])
abandonedPDFFill = abandonedPDFFill.reset_index(drop=True)

In [ ]:
#Convert all columns to string type
abandonedPDFFillString = abandonedPDFFill.applymap(str)
abandonedPDFFillString

In [ ]:
#create an empty dataframe
abandonedSingleRow = pd.DataFrame()

In [ ]:
#Storing uniqueid
abandonedSingleRow['post_uniqueId'] = abandonedPDFFillString.groupby(['post_uniqueId'])['post_uniqueId'].count().index.get_level_values('post_uniqueId')
#purchasedSingleRow is a dataframe containing post_uniqueId column

In [ ]:
#Getting browser height of each user
h1 = abandonedPDFFill.groupby(['post_uniqueId'])['post_browser_height'].apply(lambda x: x.max()).reset_index()

In [ ]:
h1['post_uniqueId'] = h1['post_uniqueId'].astype(str)
h1 = h1.sort_values(by=['post_uniqueId'])
h1 = h1.reset_index(drop=True)

In [ ]:
#Getting browser width of each user
w1 = abandonedPDFFill.groupby(['post_uniqueId'])['post_browser_width'].apply(lambda x: x.max()).reset_index()

In [ ]:
w1['post_uniqueId'] = w1['post_uniqueId'].astype(str)
w1 = w1.sort_values(by=['post_uniqueId'])
w1 = w1.reset_index(drop=True)

In [ ]:
#edit
#Get time spent by user
diff1 = abandonedPDFFill.groupby(['post_uniqueId'])['post_cust_hit_time_gmt'].apply(lambda x: x.max()-x.min()).reset_index()

In [ ]:
#sorting
diff1['post_uniqueId'] = diff1['post_uniqueId'].astype(str)
diff1 = diff1.sort_values(by=['post_uniqueId'])
diff1 = diff1.reset_index(drop=True)

In [ ]:
#add new column timespent
abandonedSingleRow['timespent'] = diff1['post_cust_hit_time_gmt']

In [ ]:
#Converting multiple rows of data into single row by joining the values using ~ for abandoned users
for cols in abandonedPDFFillString.columns:
    if (cols != 'post_uniqueId'):
        temp = abandonedPDFFillString.groupby(['post_uniqueId'])[cols].apply(lambda x: '~'.join(x)).reset_index()
        abandonedSingleRow[cols] = temp[cols]

In [ ]:
#replacing post_browser_height with single value
abandonedSingleRow['post_browser_height'] = h1['post_browser_height']

In [ ]:
#replacing post_browser_width with single value
abandonedSingleRow['post_browser_width'] = w1['post_browser_width']

In [ ]:
#Get count of number times cart was accessed
abandonedSingleRow['cartcount'] = abandonedSingleRow['post_prop14'].apply(lambda x: len(re.findall("cart", x)))

In [ ]:
#Get count of total no. of events performed by the user
abandonedSingleRow['eventcount'] = abandonedSingleRow['post_event_list'].apply(lambda x: len(re.findall(",", x))+len(re.findall("~", x))+1)

In [ ]:
#1 if user purchased 0 if abandoned
abandonedSingleRow['purchased'] = 0

In [ ]:
#Displaying newly added columns
abandonedSingleRow[['cartcount','eventcount','timespent','purchased']]

In [ ]:
#merge two dataframes of purchased and abandoned users
result = pd.concat([purchasedSingleRow,abandonedSingleRow], ignore_index=True)

In [ ]:
#Displaying the dataframe after merging
result

In [ ]:
#sorting the merged dataframe by post_uniqueId column
result = result.sort_values(by=['post_uniqueId'])
result = result.reset_index(drop=True)

In [ ]:
#Getting country of each user
countryList =[]
for index, row in result.iterrows():
    countryList.insert(index,row["post_prop2"].partition("~")[0])

In [ ]:
#Adjusting the value of country column by keeping only one value after flattening where multiple values are joined with ~
#usa~usa~usa~usa~usa is converted to usa
result["country"] = countryList

In [ ]:
#Save dataframe to a tsv file
result.to_csv("version1/allsinglerows.tsv", sep="\t", index=False)